In [ ]:
import tensorflow as tf
#tf.enable_eager_execution()

import numpy as np
import pandas as pd

from tqdm import tqdm

In [ ]:
Xs = np.load("./embed/books_train_review_en_X.npy")
Xt = np.load("./embed/books_train_review_ja_X.npy")
y = np.load("./embed/books_train_review_y.npy")

In [ ]:
def serialize_example(elsa_src, elsa_tgt, label):
    elsa_src = tf.keras.preprocessing.sequence.pad_sequences([elsa_src], dtype=elsa_src.dtype, maxlen=20)
    elsa_tgt = tf.keras.preprocessing.sequence.pad_sequences([elsa_tgt], dtype=elsa_tgt.dtype, maxlen=50)
    #assert elsa_src.shape == elsa_tgt.shape == (1, 20, 2248)
    feature = {
        "src":   tf.train.Feature(float_list=tf.train.FloatList(value=elsa_src.flatten().tolist())),
#        "src_len":  tf.train.Feature(int64_list=tf.train.Int64List(value=[len(elsa_src)])),
        "tgt":   tf.train.Feature(float_list=tf.train.FloatList(value=elsa_tgt.flatten().tolist())),
#        "tgt_len": tf.train.Feature(int64_list=tf.train.Int64List(value=[len(elsa_tgt)])),
        "label":  tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example.SerializeToString()

In [ ]:
with tf.python_io.TFRecordWriter("test.tfrecord") as writer:
    for i in tqdm(range(len(y))):
        example = serialize_example(Xs[i], Xt[i], y[i])
        writer.write(example)

In [ ]:
def parse_example(example):
    features = tf.parse_single_example(
        example,
        features={
            'src': tf.io.VarLenFeature(tf.float32),
#            'src': tf.FixedLenFeature([], tf.float32, default_value=0.0),
#            'src_len': tf.FixedLenFeature([], tf.int64, default_value=0),
            'tgt': tf.io.VarLenFeature(tf.float32),
#            'tgt': tf.FixedLenFeature([], tf.float32, default_value=0.0),
#            'tgt_len': tf.FixedLenFeature([], tf.int64, default_value=0),            
            'label': tf.FixedLenFeature([], tf.int64, default_value=0),
        })    
    src = tf.reshape(features["src"].values, (20, -1)) 
    tgt = tf.reshape(features["tgt"].values, (50, -1)) 
    label = features["label"]
    return {"src": src, "tgt": tgt, "label": label}

In [ ]:
dataset = tf.data.TFRecordDataset(['test.tfrecord'])

In [ ]:
iterator = dataset.map(parse_example).make_one_shot_iterator()
get_next = iterator.get_next()
with tf.Session() as sess:
     data = sess.run(get_next)
s_depth = data["src"].shape[1]
t_depth = data["tgt"].shape[1]
[s_depth, t_depth]

In [ ]:
iterator = dataset.map(parse_example).make_one_shot_iterator()
get_next = iterator.get_next()
with tf.Session() as sess:
    i = 0
    try:
        while True:
            sess.run(get_next)
            i += 1
    except tf.errors.OutOfRangeError:
        pass
i

In [ ]:
iterator = dataset.map(parse_example).shuffle(1000).batch(32).repeat(1).make_one_shot_iterator()
get_next = iterator.get_next()
with tf.Session() as sess:
    try:
        while True:
            data = sess.run(get_next)
            print(data)
            break
    except tf.errors.OutOfRangeError:
        pass

In [ ]:
data["src"].shape

In [ ]:
np.all(data["src"].numpy().reshape((1, 20, -1)) ==  tf.keras.preprocessing.sequence.pad_sequences([Xs[0]], dtype="float32", maxlen=20))

In [ ]:
for data in dataset.shuffle(1000).batch(32).take(1):
    print(data)

In [ ]:
data["src"].numpy().shape

In [ ]:
data["tgt"].numpy().shape

In [ ]:
data["label"].numpy().shape

In [ ]:
iterator = dataset.shuffle(1000).batch(32).make_one_shot_iterator()

In [ ]:
for i, data in enumerate(dataset.repeat(2).shuffle(1000).batch(32)):
    pass

In [ ]:
data["label"].numpy()

In [ ]:
RepeatDataset = type(tf.data.Dataset().repeat())

In [ ]:
labels = []
for data in dataset.repeat(1):
    labels.append(data["label"].numpy())

In [ ]:
np.array(labels)

In [ ]:

record_iterator = tf.python_io.tf_record_iterator(path="test.tfrecord")
for string_record in record_iterator:
    example = tf.train.Example()
    example.ParseFromString(string_record)
    break

np.array(example.features.feature["src"].float_list.value).reshape(20, -1).shape